# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import AlexNet, AlexNet_Weights
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import warnings
warnings.filterwarnings("ignore")

### SELECT DEVICE ###
# GPU device configuration
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
  print('Using GPU')
elif torch.backends.mps.is_available():
  DEVICE = torch.device('mps')
  print('Using MPS')
else:
  DEVICE = torch.device('cpu')
  print('Using CPU')


# Dataloader

In [ ]:
### Build Cats vs Dogs Dataset ###
PATH_TO_DATA = "./data/cats_vs_dogs/train/"

### DEFINE TRANSFORMATIONS ###
normalizer = transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                  std = [0.229, 0.224, 0.225]) ### IMAGENET MEAN/STD ###
train_transforms = transforms.Compose([
                                        transforms.Resize((224,224)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        normalizer
                                      ])


dataset = ImageFolder(PATH_TO_DATA, transform=train_transforms)

train_samples, test_samples = int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, lengths=[train_samples, test_samples])

print(dataset)
print(train_samples, test_samples)
print(train_dataset, val_dataset)

# Model training function

In [20]:
def train(model, device, epochs, optimizer, loss_fn, batch_size, trainloader, valloader):
    log_training = {"epoch": [],
                    "training_loss": [],
                    "training_acc": [],
                    "validation_loss": [],
                    "validation_acc": []}

    for epoch in range(1, epochs + 1):
        print(f"Starting Epoch {epoch}")
        training_losses, training_accuracies = [], []
        validation_losses, validation_accuracies = [], []

        for image, label in tqdm(trainloader):
            image, label = image.to(DEVICE), label.to(DEVICE)
            optimizer.zero_grad()
            out = model.forward(image)
        
            ### CALCULATE LOSS ##
            loss = loss_fn(out, label)
            training_losses.append(loss.item())

            ### CALCULATE ACCURACY ###
            predictions = torch.argmax(out, axis=1)
            accuracy = (predictions == label).sum() / len(predictions)
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()

        for image, label in tqdm(valloader):
            image, label = image.to(DEVICE), label.to(DEVICE)
            with torch.no_grad():
                out = model.forward(image)

                ### CALCULATE LOSS ##
                loss = loss_fn(out, label)
                validation_losses.append(loss.item())

                ### CALCULATE ACCURACY ###
                predictions = torch.argmax(out, axis=1)
                accuracy = (predictions == label).sum() / len(predictions)
                validation_accuracies.append(accuracy.item())

        training_loss_mean, training_acc_mean = np.mean(training_losses), np.mean(training_accuracies)
        valid_loss_mean, valid_acc_mean = np.mean(validation_losses), np.mean(validation_accuracies)

        log_training["epoch"].append(epoch)
        log_training["training_loss"].append(training_loss_mean)
        log_training["training_acc"].append(training_acc_mean)
        log_training["validation_loss"].append(valid_loss_mean)
        log_training["validation_acc"].append(valid_acc_mean)

        print("Training Loss:", training_loss_mean) 
        print("Training Acc:", training_acc_mean)
        print("Validation Loss:", valid_loss_mean)
        print("Validation Acc:", valid_acc_mean)
        print("=====================================\n")
        
    return log_training, model

## Load PreTrained Weights but Only Train the Final Classifier Layer

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model.fc
model.fc = nn.Linear(2048, 2)

# Check the name of all the parameters
for name, param in model.named_parameters():
  print(name)
  if "fc" not in name:
    param.requires_grad_(False) # Inplace turn of gradient updates

In [ ]:
total_parameters = 0
for name, params in model.named_parameters():
  num_params = int(torch.prod(torch.tensor(params.shape)))
  print(name, ":", params.shape, "Num Parameters:", num_params)
  total_parameters += num_params

print("------------------------")
print("Total Parameters in Model", total_parameters)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model.fc = nn.Linear(2048, 2)

# Check the name of all the parameters
for name, param in model.named_parameters():
  print(name)
  if "fc" not in name:
    param.requires_grad_(False) # Inplace turn of gradient updates

model = model.to(DEVICE)

### MODEL TRAINING INPUTS ###
epochs = 5
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

### BUILD DATALOADERS ###
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


random_init_logs, model = train(model=model,
                                device=DEVICE,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=trainloader,
                                valloader=valloader)